In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1249799/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_9_49_9,0.999929,0.600442,0.578855,0.999875,0.000023,0.237195,0.295021,0.000036,0.001665,0.004842,1.000036,0.004879,791.321976,1895.322740,"Hidden Size=[24, 12], regularizer=0.02, learni..."
1,model_9_49_8,0.999928,0.600522,0.578838,0.999874,0.000024,0.237147,0.295033,0.000036,0.001673,0.004881,1.000036,0.004919,791.289797,1895.290560,"Hidden Size=[24, 12], regularizer=0.02, learni..."
2,model_9_49_5,0.999927,0.600765,0.578912,0.999886,0.000024,0.237003,0.294981,0.000033,0.001678,0.004911,1.000037,0.004950,791.264826,1895.265590,"Hidden Size=[24, 12], regularizer=0.02, learni..."
3,model_9_49_7,0.999927,0.600632,0.578928,0.999874,0.000024,0.237082,0.294969,0.000036,0.001680,0.004916,1.000037,0.004954,791.261276,1895.262039,"Hidden Size=[24, 12], regularizer=0.02, learni..."
4,model_9_49_4,0.999926,0.600857,0.578949,0.999888,0.000024,0.236948,0.294955,0.000032,0.001681,0.004940,1.000037,0.004978,791.241781,1895.242544,"Hidden Size=[24, 12], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988,model_3_0_1,-0.106519,-0.033971,0.169831,0.066846,0.367324,0.613810,0.581551,0.374176,0.300976,0.606073,-0.784263,0.610789,100.003021,240.512209,"Hidden Size=[12], regularizer=0.02, learning_r..."
4989,model_1_0_0,-0.113352,-0.034586,-0.117548,-0.074422,0.369592,0.614175,0.782865,0.319571,0.299450,0.607941,-0.282343,0.612672,35.990709,84.738795,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4990,model_3_0_0,-0.143594,-0.074621,0.147029,0.026268,0.379632,0.637942,0.597524,0.390447,0.306432,0.616143,-0.844045,0.620937,99.937108,240.446296,"Hidden Size=[12], regularizer=0.02, learning_r..."
4991,model_4_0_0,-0.185519,-0.229620,-0.035189,-0.330124,0.393549,0.729955,0.725171,0.396979,0.317480,0.627335,-3.779123,0.632217,195.865099,474.015940,"Hidden Size=[24], regularizer=0.02, learning_r..."
